<a href="https://colab.research.google.com/github/chrisrichardson/linear-algebra/blob/main/01_Linear_Algebra_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Import the libraries we are going to use
import numpy as np
import numba

# Matrices and vectors

* vector = 1D: e.g. $(1.0, 2.2, -3.0, 4.0)$
* matrix = 2D, e.g. $\left(\begin{matrix} 0.01 & 2.2 \\ -3.3 & 4.1 \end{matrix}\right)$
* tensor = N-dimensional

All of these are supported by `numpy`.


## Vectors

Note that a vector can be a row or a column, i.e. the shape can be `(1, N)` or `(N, 1)` - or just `(N,)`. The actual *value data* in memory is the same for all of these.

In [7]:
# Vector
b = np.array([1.0, 2.0, 3.0])
print(b, b.shape)
print()

# Row vector
b = np.array([[1.0, 2.0, 3.0]])
print(b, b.shape)
print()

# Column vector
b = np.array([[1.0], [2.0], [3.0]])
print(b, b.shape)
print()

[1. 2. 3.] (3,)

[[1. 2. 3.]] (1, 3)

[[1.]
 [2.]
 [3.]] (3, 1)



### dtype

A `numpy` array has a data type (or `dtype`) associated with it, which is usually a floating point type (e.g. 64-bit or "double" precision). You can set it or check it.

In [12]:
# Looking at dtypes
b = np.array([1.0, 2.0, 5.0, 8.0])
print(b, b.dtype)

b = np.array([1, 2, 3])
print(b, b.dtype)

b = np.array([1, 2, 3], dtype=np.float32)
print(b, b.dtype)


[1. 2. 5. 8.] float64
[1 2 3] int64
[1. 2. 3.] float32


### Operations on vectors

dot product: `a.dot(b)`

axpy: `w = a * x + y`

norm: `np.linalg.norm(a)`

In [23]:
a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 0.5, -1.0])

# Dot product
print(a.dot(b))

# Norm
print(np.linalg.norm(a), np.linalg.norm(b))

# axpy
c = a + b
print(c)

# More unusual operation - pointwise multiplication:
c = a * b
print(c)


0.0
3.7416573867739413 2.29128784747792
[3.  2.5 2. ]
[ 2.  1. -3.]


## Matrices

A matrix is a two-dimensional array which can be used for linear algebra operations.

### Matrices in numpy

Matrices behave in numpy in a similar way to vectors.

**Storage**

The data is generally stored in contiguous memory, either *Row Major* (C-style) or *Column Major* (FORTRAN-style).
Every entry is stored (even if it is zero). This is called **dense** storage.

In [ ]:
A = np.array([[1, -2, 3],
              [3, 1, 5],
              [3, 4, 1]], dtype=np.float64)
print(A)
print(A.shape)
print(A.dtype)
print(A.flags)

[[ 1. -2.  3.]
 [ 3.  1.  5.]
 [ 3.  4.  1.]]
(3, 3)
float64
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



In [29]:
# The default order is "C" (Row Major) but we can change the storage order

A = np.array([[1, -2, 3],
              [3, 1, 5],
              [3, 4, 1]], dtype=np.float32, order='F')
print(A)
print(A.shape)
print(A.dtype)
print(A.flags)

[[ 1. -2.  3.]
 [ 3.  1.  5.]
 [ 3.  4.  1.]]
(3, 3)
float32
  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



### Matrix-vector product

Numpy supports matrix-vector product using the `@` operator, but there are several other ways we could do it. We will explore the different ways of doing "MatVec" and also try `numba` to generate efficient code.
`numba` compiles the Python code in a function to machine code, so it runs much faster.

In [38]:
# First, let's use the numpy built-in method

A = np.array([[1, -2, 3],
              [3, 1, 5],
              [3, 4, 1]], dtype=np.float64)
b = np.array([1, 2, 3], dtype=np.float64)

print(A)
print(b)
print(A @ b)

[[ 1. -2.  3.]
 [ 3.  1.  5.]
 [ 3.  4.  1.]]
[1. 2. 3.]
[ 6. 20. 14.]


In [39]:
# We can re-implement this by hand by doing a dot product of each row of `A` with the vector `b`

def matvec0(A, b):
  y = np.empty(A.shape[0])
  for i in range(A.shape[0]):
    y[i] = np.dot(A[i, :], b[:])
  return y

# It should give the same answer!
print(matvec0(A, b))
print(A @ b)

[ 6. 20. 14.]
[ 6. 20. 14.]


In [48]:
# But of course, it is much slower. We can use the built-in `%%timeit` to find out.
# First let's make a bigger matrix and vector
N = 1000
A = np.random.random((N, N))
b = np.random.random(N)

In [49]:
%%timeit -n 100
A @ b

411 µs ± 97.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
%%timeit -n 100
matvec0(A, b)

2.2 ms ± 88.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
# We can compile our Python code using numba
matvec0_numba = numba.njit(matvec0)

In [62]:
# Rerun this cell a few times: the first iteration is slower, because it needs to compile.
%%timeit -n 100
matvec0_numba(A, b)

421 µs ± 25.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Exercise

Use the rest of this notebook to experiment with MatVec.
Here are some things to try:

* change the matrix `A` from Row Major(`'C'`) to Column Major(`'F'`)
* use different `numpy` functions (`dot` or `einsum` or `matmul` to do the MatVec) - does it matter?
* write a different matvec "`matvec1`" which does not use a reduction, but accumulates the solution instead. Check it gives the same answer.
* compile `matvec1` using `numba`

(hint: the `matvec1` operation should look like `y += A[:, j] * b[j]`)

For each case, try timing it using `%%timeit` - which is faster for `matvec0`, Row Major or Column Major? And for `matvec1`?

